In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image


In [4]:
# (Später ersetzen) Hauptpfad zum entpackten Mapillary Vistas Dataset
MAPILLARY_DATASET_PATH = "PFAD_ZUM_MAPILLARY_VISTAS"  

# (Später ersetzen) Pfad zu den gelabelten Ground-Truth-Bildern
LABELS_PATH = os.path.join(MAPILLARY_DATASET_PATH, "PFAD_ZU_LABEL_BILDERN")

# (Später ersetzen) Pfad zum Speicherort für die umgelabelten Bilder
OUTPUT_LABELS_PATH = os.path.join(MAPILLARY_DATASET_PATH, "PFAD_ZU_NEU_LABEL_BILDERN")

# Mapillary → Cityscapes Label-Mapping (noch nicht befüllt, das kommt in einem späteren Schritt)
LABEL_MAPPING = {}

# Farben für die Visualisierung der Labels (RGB Dictionaries, später definieren)
ORIGINAL_COLOR_MAP = {}
NEW_COLOR_MAP = {}


In [5]:
MAPILLARY_TO_CITYSCAPES = {
    7: 'road',
    8: 'sidewalk',
    11: 'building',
    12: 'wall',
    13: 'fence',
    17: 'pole',
    19: 'traffic light',
    20: 'traffic sign',
    21: 'vegetation',
    22: 'terrain',
    23: 'sky',
    24: 'person',
    25: 'rider',
    26: 'car',
    27: 'truck',
    28: 'bus',
    31: 'train',
    32: 'motorcycle',
    33: 'bicycle',
    
    # Neue Klasse für Fahrbahnmarkierungen (Lane Markings)
    200: 'lane marking',

    # Alle nicht benötigten Klassen werden als 'unlabeled' behandelt
    0: 'unlabeled'
}